# python pandas

In [2]:
import os
import sys
import sysconfig
from pathlib import Path
import shutil
import subprocess
import tempfile
import json

import re
import difflib
import pypdf
import docx
import chardet
import datetime
import dateutil

import pandas as pd
import numpy as np

from pprint import pprint

## chardet

1. Used for detecting the encoding of CSV files.
2. For xls and xlsx files, encoding issues usually don't require much attention because these files use UTF-8 as the standard encoding.
3. For csv files, it is often necessary to detect the encoding before reading the file.
4. Older xls files may have encoding issues, which can be addressed as they arise.

In [8]:
# Simple usage of chardet
## The file needs to be opened in binary mode before using the detect method.
## ASCII encoded files can be read directly using UTF-8.

with open('originalxls.csv', 'rb') as f:
    result = chardet.detect(f.read())
    pprint(result)

{'confidence': 1.0, 'encoding': 'ascii', 'language': ''}


## pandas


### Reading file 

In [25]:
# csv

myPdCsv = pd.read_csv('originalxls.csv', encoding='utf-8')
print(myPdCsv)

        ?? column1 column2 column3
0  record1     str      %d      %f
1  record2      \n    str2     NaN
2  record3      \r     NaN    str3


In [24]:
# excel
## No need to specify encoding

myPdExcel1 = pd.read_excel('originalxls.xlsx', sheet_name='Sheet1')
print(myPdExcel1)

        表头  column1  column2  column3
0  record1        1        2        3
1  record2        4        5        6
2  record3        7        8        9


#### Read files on demand

In [33]:
# Read all sheets in an xlsx file
## Setting the sheet_name parameter to None will read all sheets.
## If the sheet_name parameter is not specified, the first sheet will be read by default.
## The result is a dictionary where the key is the sheet name and the value is a dataframe.

myPdExcelList = pd.read_excel('originalxls.xlsx', sheet_name=None)
pprint(myPdExcelList,indent=4)
print()
print(myPdExcelList['Sheet1'])

{   'Sheet1':         表头  column1  column2  column3
0  record1        1        2        3
1  record2        4        5        6
2  record3        7        8        9,
    'Sheet2':         表头  column1  column2  column3
0  record1       11     21.0     31.0
1  record2       14      NaN      NaN
2  record3       17      NaN      NaN,
    'Sheet3':         表头 column1 column2 column3
0  record1     str      %d      %f
1  record2      \n    str2     NaN
2  record3      \r     NaN    str3}

        表头  column1  column2  column3
0  record1        1        2        3
1  record2        4        5        6
2  record3        7        8        9


In [21]:
# Specify column names and index column
# Both the header and index_col parameters accept index values
## The header parameter specifies the row for the column headers, and index_col specifies the index column. These settings apply to all sheets being read.


myPdExcelList2 = pd.read_excel('originalxls.xlsx',sheet_name=None, header=0, index_col=0)
pprint(myPdExcelList2['Sheet1'])

         column1  column2  column3
表头                                
record1        1        2        3
record2        4        5        6
record3        7        8        9


In [2]:
# By specifying the names parameter, you can define column names
## The header parameter should be set to None.


myPdExcelList3 = pd.read_excel('originalxls.xlsx', header=None, names=['a', 'b', 'c', 'd'], index_col=None)
print(myPdExcelList3)

         a        b        c        d
0       表头  column1  column2  column3
1  record1        1        2        3
2  record2        4        5        6
3  record3        7        8        9


#### tips

In [29]:
## Note: If the column names are set incorrectly, it may lead to unpredictable input and output.


myPdExcelList3 = pd.read_excel('originalxls.xlsx', header=None, names=['a', 'b', 'c'], index_col=None) # The original table has 4 columns, but one column name was not specified here.


In [30]:
## The first column has been used as the index, causing the index_col parameter to be ineffective.


print(myPdExcelList3)

               a        b        c
表头       column1  column2  column3
record1        1        2        3
record2        4        5        6
record3        7        8        9


#### json

The orient parameter specifies the format of the JSON, commonly using split, records, index, columns, values:

Options:
'split' : dict like {index -> [index], columns -> [columns], data -> [values]}  
例：{'index': [...], 'columns': [...], 'data': [...]}

'records' : list like [{column -> value}, ... , {column -> value}]

'index' : dict like {index -> {column -> value}}

'columns' : dict like {column -> {index -> value}}

'values' : just the values array

'table' : dict like {'schema': {schema}, 'data': {data}}


In [10]:
# Read json

pprint(json.load(open('example_data.json')))

{'departments': [{'manager': 'John Doe', 'name': 'Sales'},
                 {'manager': 'Jane Smith', 'name': 'HR'},
                 {'manager': 'Jim Doe', 'name': 'IT'}],
 'employees': [{'age': 30, 'department': 'Sales', 'name': 'John Doe'},
               {'age': 25, 'department': 'HR', 'name': 'Jane Doe'},
               {'age': 35, 'department': 'IT', 'name': 'Jim Doe'}]}


In [13]:
myPdJson = pd.read_json('example_data.json', orient='index')
print(myPdJson)

                                                             0  \
employees    {'name': 'John Doe', 'age': 30, 'department': ...   
departments           {'name': 'Sales', 'manager': 'John Doe'}   

                                                             1  \
employees    {'name': 'Jane Doe', 'age': 25, 'department': ...   
departments            {'name': 'HR', 'manager': 'Jane Smith'}   

                                                             2  
employees    {'name': 'Jim Doe', 'age': 35, 'department': '...  
departments               {'name': 'IT', 'manager': 'Jim Doe'}  


In [46]:
# It was found that the JSON data is nested, which doesn't quite match the orient parameter.
## First convert the JSON data into a dictionary, then create dataframes separately.

# Load JSON data as a Python dictionary

with open('example_data.json', 'r') as file:
    data = json.load(file)

# Convert a specific part of the data into a DataFrame
employees_df = pd.DataFrame(data['employees'])
departments_df = pd.DataFrame(data['departments'])

# Display the result
print("Employees DataFrame:")
print(employees_df)
print("\nDepartments DataFrame:")
print(departments_df)


Employees DataFrame:
       name  age department
0  John Doe   30      Sales
1  Jane Doe   25         HR
2   Jim Doe   35         IT

Departments DataFrame:
    name     manager
0  Sales    John Doe
1     HR  Jane Smith
2     IT     Jim Doe


#### Accessing SQL

Can use `pandas.read_sql` and pass a **query string** and a **database engine** to read data from the database.


### Output data to Excel

The focus is on exporting data by sheets to Excel.

Prototype:
DataFrame.to_excel(excel_writer, *, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, inf_rep='inf', freeze_panes=None, storage_options=None, engine_kwargs=None)

In [49]:
# Specify sheet names when exporting to Excel
# Use pd.ExcelWriter to manage this process

with pd.ExcelWriter('multiSheetOutput.xlsx') as writer:
    employees_df.to_excel(writer, sheet_name='employees', index=False)
    departments_df.to_excel(writer, sheet_name='departments', index=False)

## 数据类型和结构

In [2]:
myPdExcelList = pd.read_excel('originalxls.xlsx', sheet_name=None)
pprint(myPdExcelList,indent=4)
sheet1 = myPdExcelList['Sheet1']
sheet2 = myPdExcelList['Sheet2']
sheet3 = myPdExcelList['Sheet3']

{   'Sheet1':         表头  column1  column2  column3
0  record1        1        2        3
1  record2        4        5        6
2  record3        7        8        9,
    'Sheet2':         表头  column1  column2  column3
0  record1       11     21.0     31.0
1  record2       14      NaN      NaN
2  record3       17      NaN      NaN,
    'Sheet3':         表头 column1 column2 column3
0  record1     str      %d      %f
1  record2      \n    str2     NaN
2  record3      \r     NaN    str3}


### Data Types

can use DataFrame_object.dtypes to view data types, which are displayed by column.

Main types include:

    object: Used for mixed data objects. category: A subclass of object, representing variables with a fixed number of possible values, similar to enums.
    Most pandas objects correspond to numpy types, except for object and category.
    Numpy integers do not support NaN, but pandas.Int64 (note the uppercase) supports NaN.
    Int64, Int32, UInt64, etc., are pandas-extended integer types that can support NaN.
    pandas.int8, pandas.int16, pandas.int32, "pandas.int64" (double quotes indicate the default type).
    pandas.float32, "pandas.float64".
    datetime64[ns, tz], "datetime64[ns]" uses nanoseconds by default, also from numpy. "timedelta[ns]" also comes from numpy. Note that since data types can be specified using dtype, datetime64[ns, tz] is useful.
    Example: dtype: datetime64[ns, Asia/Shanghai].
    It is recommended to use StringDtype for strings, instead of the old object.
    Strings can be processed using standard string methods. Common ones include replace(), and don't forget about map() (for series), apply() (works for both series and DataFrame, and can specify rows or columns), and lambda expressions.
    Add str before processing strings, e.g., df['A'].str.replace('a', 'b').
    Use methods like .astype("float32") for explicit type conversion.
    The Dataframe.infer_objects() method can be used to infer the type of object for convenience.
    Missing values are of type np.nan.

In [3]:
# Check the data type of each column

sheet1.dtypes

表头         object
column1     int64
column2     int64
column3     int64
dtype: object

In [6]:
# You can directly specify the data type of a variable (even though the variable contains no data).


a = pd.DatetimeTZDtype("ns",tz="Asia/Shanghai")
print(type(a))
pd.Series(['20200501 22:23:22.3432'], dtype=a).dtypes

<class 'pandas.core.dtypes.dtypes.DatetimeTZDtype'>


datetime64[ns, Asia/Shanghai]

### series

1. When specifying `index=`, it must match the length of the data (it can be an iterator).
2. If the `index` exceeds the data length, the `data` will be automatically filled.
3. You can specify `name`.


In [17]:
# If the index exceeds the data length, it will be automatically filled.
## Also, specify the name.

pd.Series(5.0, index=["a", "b", "c", "d", "e"], name="test")

a    5.0
b    5.0
c    5.0
d    5.0
e    5.0
Name: test, dtype: float64

### dataframe

#### Series, DataFrame, and Matrix

1. First, a DataFrame is made up of Series, so each column can be extracted separately to form an array[].
2. Don’t be confused by how numpy and pandas represent matrices—they both default to forming matrices by combining column vectors and provide methods to extract columns. You only need to focus on how to manipulate them.
3. If the matrix isn’t in the desired form, you can use `transpose()` or simply `T` to transpose it.


In [15]:
import pandas as pd

# Column Vector
col_vector1 = [1, 2, 3]
col_vector2 = [4, 5, 6]

# Combine into a DataFrame, with each list becoming a column.

df = pd.DataFrame({'col1': col_vector1, 'col2': col_vector2})
df2 = pd.DataFrame([col_vector1, col_vector2], columns=['a', 'b', 'c'])
df2t = df2.T
print(df)
print()
print(df2)
print()
print(df2t)

   col1  col2
0     1     4
1     2     5
2     3     6

   a  b  c
0  1  2  3
1  4  5  6

   0  1
a  1  4
b  2  5
c  3  6


In [ ]:
# Intuitive construction method

df = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6],
                    'C': [7, 8, 9]})

#### Axis Direction

1. In numpy, since it can handle multi-dimensional arrays, the axis parameter can be greater than or equal to 2. The order of the axis parameter typically represents the **scope of the unit**, from outer to inner.
2. For example, in a three-dimensional array, `axis=0` represents the largest scope, which is each array itself, `axis=1` represents the rows within each array, and `axis=2` represents the elements within each row of the array.
3. In a two-dimensional array, `axis=0` refers to rows, which looks like you are processing **columns**; `axis=1` refers to elements within each row, which looks like you are processing **rows**.
4. Think about it: myArray[0][1] naturally refers to the **first row** and the **second element**. Therefore, `axis=0` means **processing in the direction extending along rows**, and `axis=1` means processing in the direction of elements, **extending along the elements**.


### Copy

1. If you need another copy, don’t assign it directly to another variable, as this will only **link** the two variables.
2. Use pandas’ built-in `copy()` method. You can specify the `deep=` parameter, which is `True` by default.


## Structural Operations

### Value Selection and Slicing Rules

General rule: **Single value** selection uses single brackets, **multiple value** selection uses double brackets. Slicing is generally allowed wherever you can pass an index, and both names and numbers can be used for slicing.

1. `df` allows column selection using `df['column']` or `df.column`, returning a series.
2. To select multiple columns in `df`, double brackets are required: `df[['column1', 'column2']]`, which returns a **dataframe**.
   1. When swapping columns, `df[['B', 'A']] = df[['A', 'B']]` is quite useful.
3. Slicing with `df[]` actually returns rows, like `df[0:1]`, `df['a':'b']`.
   1. `s[]` and `s.loc[]` are basically the same.
4. My suggestion is to use `loc` and `iloc` for DataFrame indexing, bury those who directly select values, and half-bury those who select columns using attributes.
   1. Writing like `df[:,]` is pretty much a death sentence.
5. Use `loc[]` to select by name and `iloc[]` to select by position. The order is `[row, column]` (**emphasizing again, `loc` and `iloc` should use [] brackets**).
   1. `loc` and `iloc` support coordinate indexing: `df.loc['index', 'column']`.
   2. To select multiple values: `df.iloc[[1, 3, 5], [1, 3]]`: retrieves the intersections of rows 1, 3, 5 and columns 1, 3.
   3. `iloc` supports slicing: `df.iloc[0:2, 0:2]`; `loc` also supports slicing: `df.loc['a':'b', 'A':'B']`.
   4. Since `loc[]` supports boolean indexing, you can filter like this: `loc[s.apply(lambda x: x % 2 == 0)]`.
      1. `s[s.isin([2, 4, 6])]` is very useful.
   5. They both support passing a function, so you can retrieve values like this: `df.loc[lambda df: df['A'] > 0]`.
6. Reverse and step slicing uses the third position in `:`: `df.iloc[::-1]`, `df.iloc[::-2]`.
7. `at` and `iat` are used for selecting single values by passing a coordinate tuple. They are essentially similar to `loc` and `iloc`, but a bit faster.
8. `NaN` is less than 0.
9. Use `.drop()` to remove specific rows or columns, and you can specify the `axis` parameter.



#### series and pd example

In [2]:

data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
    'Salary': [60000, 70000, 80000, 90000, 100000]
}

df = pd.DataFrame(data, index=['A', 'B', 'C', 'D', 'E'])

print(df)


      Name  Age         City  Salary
A    Alice   25     New York   60000
B      Bob   30  Los Angeles   70000
C  Charlie   35      Chicago   80000
D    David   40      Houston   90000
E      Eve   45      Phoenix  100000


In [29]:
# series

s = df.City
print(s)
print()
print(s["E"])
print()
print(s.iloc[4])

A       New York
B    Los Angeles
C        Chicago
D        Houston
E        Phoenix
Name: City, dtype: object

Phoenix

Phoenix


In [4]:
# DataFrame
## Mainly to demonstrate some confusing methods for directly selecting rows and columns.

print(df[:2:-1])
print()
print(df[["Name","City"]])

    Name  Age     City  Salary
E    Eve   45  Phoenix  100000
D  David   40  Houston   90000

      Name         City
A    Alice     New York
B      Bob  Los Angeles
C  Charlie      Chicago
D    David      Houston
E      Eve      Phoenix


#### Conditional Selection or Deletion (Masking, where)

1. Primarily use [], then where(), and lastly query().
2. When filtering data, first check if the function allows passing the axis parameter (where can).
3. You can still use apply() in combination with lambda expressions.

In [22]:
# Example 
df_num = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

print(df_num)
print()

     A   B   C   D
0   93  93  36  47
1   83  54  97  69
2   79  35  61  20
3   23  26  95  17
4    2  40  37  36
..  ..  ..  ..  ..
95  75   9  77  49
96   2  60   7  11
97  71  93  84  69
98  90  40  42  28
99   5  34  68  69

[100 rows x 4 columns]



In [23]:
# return boolean series

print(df_num['A'] > 50)

0      True
1      True
2      True
3     False
4     False
      ...  
95     True
96    False
97     True
98     True
99    False
Name: A, Length: 100, dtype: bool


In [23]:
# Return a boolean DataFrame
## Understand the syntax, as this is the foundation for many conditional selections.
## It's equivalent to .where()

print(df_num > 50)

        A      B      C      D
0   False  False   True   True
1   False   True  False  False
2    True  False   True  False
3    True  False  False  False
4   False   True  False  False
..    ...    ...    ...    ...
95  False   True   True   True
96   True  False  False  False
97  False  False  False   True
98  False   True  False   True
99   True  False   True  False

[100 rows x 4 columns]


In [27]:
# Usage of where
## where can directly replace the original values.
## You can also pass the axis parameter.

print(df_num.where(df_num > 50, 0))

     A   B   C   D
0    0   0  61  86
1    0  85   0   0
2   76   0  57   0
3   97   0   0   0
4    0  99   0   0
..  ..  ..  ..  ..
95   0  93  97  69
96  57   0   0   0
97   0   0   0  76
98   0  72   0  51
99  77   0  74   0

[100 rows x 4 columns]


In [24]:
# With the boolean DataFrame as a mask
## It will return a DataFrame that only shows data greater than 50.

print(df_num[df_num > 50])
print()

       A     B     C     D
0    NaN   NaN  61.0  86.0
1    NaN  85.0   NaN   NaN
2   76.0   NaN  57.0   NaN
3   97.0   NaN   NaN   NaN
4    NaN  99.0   NaN   NaN
..   ...   ...   ...   ...
95   NaN  93.0  97.0  69.0
96  57.0   NaN   NaN   NaN
97   NaN   NaN   NaN  76.0
98   NaN  72.0   NaN  51.0
99  77.0   NaN  74.0   NaN

[100 rows x 4 columns]



In [20]:
# Select rows where column A is greater than 50

print(df_num[df_num.A > 50])

     A   B   C   D
4   82  90  38  53
5   84  19  44  12
6   52  78  87  82
7   66  42  13  26
9   93   3  60  50
10  60  31  57   2
11  95  58  28  33
14  87  96  70  67
15  55  87  37  90
16  91  46  40   3
18  89  80  32   0
19  97  94  73  57
20  57  65   5  68
22  78  11  45  66
25  90  67  18  76
26  80   1  76  77
27  95  77   7  19
29  75  17  15  15
30  78  76  69  68
31  70  84  74  32
32  77  52  85   9
34  62  89  66  58
37  60  25  23  75
38  53  48  29  68
40  54  83  33  19
42  93  46  16  13
43  96  18  41  85
44  92  68  70  54
46  64  52  51  47
50  61  78  14  17
51  67  40  71  96
55  64  56  53  82
56  81  79  43  27
58  86  61  84  20
59  81  57  19  96
60  57  95  78  87
61  74  88  40  90
63  60  51  97  75
64  74  13  95  35
65  88  26  23  37
66  57  89  10  62
73  54  18  38  46
74  84  60  44  18
75  91  92  72  17
77  79  82  95  14
78  64  51  44  38
79  53   6   1  63
80  97  16  16  44
81  83  22  95  38
82  94  49  22  52
83  58  28  48   4
87  96  75  

### Best Practices for Data Types

For columns with specific type requirements, set the data types beforehand and then construct the DataFrame.


In [9]:

A_type = "Int64"
B_type = pd.CategoricalDtype(categories=["b", "c", "d"], ordered=True)
C_type = "string"
D_type = "boolean"

df = pd.DataFrame({ 
                   "A": pd.Series([1, 2, 3, 4], dtype=A_type), 
                   "B": pd.Series(["b", "c", "d", "b"], dtype=B_type), 
                   "C": pd.Series(["foo", "bar", "foo", "bar"], dtype=C_type), 
                   "D": pd.Series([True, False, True, False], dtype=D_type) 
                   })

print(df.dtypes)

# What to do if the type is incorrect?
# Use astype, or assign the correct value directly.


A             Int64
B          category
C    string[python]
D           boolean
dtype: object


### Categorical

In [3]:
# create a category type

s = pd.Series(["a", "b", "c", "a"], dtype="category")
s

0    a
1    b
2    c
3    a
dtype: category
Categories (3, object): ['a', 'b', 'c']

In [4]:
# Use the Categorical class to define the structure of categorical data
# You can remove unnecessary categories.

df = pd.DataFrame({"A": list("abca"), "B": list("bccd")}, dtype="category")
df["A"] = pd.Categorical(df["A"], categories=["b", "c", "d"], ordered=True)

## df = pd.DataFrame(pd.Categorical("A": list("abca"), categories=["b", "c", "d"], ordered=True)
## df["A"].astype(CategoricalDtype(categories=['foo', 'bar'], ordered=True))
print(df.dtypes)
print()
print(df)

A    category
B    category
dtype: object

     A  B
0  NaN  b
1    b  c
2    c  c
3  NaN  d


### Binning with cut and qcut

Data can be divided into discrete intervals, where you can specify the boundaries of the intervals or the number of intervals.  
Using `qcut` instead of `cut` allows for dividing intervals based on percentiles rather than fixed values.

Special Parameters:
1. `ordered = True` is effective when passing a series.
2. `right` defaults to True, meaning left-open and right-closed intervals; False means left-closed and right-open intervals.
3. `include_lowest` defaults to False, True ensures that the left endpoint of the first interval is closed, including the minimum value.
4. `retbins` defaults to False, True means it will return the bin edges as an array.


In [14]:
df_num = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

print(df_num)
print()

     A   B   C   D
0   23  29  37  92
1   74  31  23  71
2   19  26   0  71
3   58  67  37  77
4   33  21  61  33
..  ..  ..  ..  ..
95  73  86  26  54
96  82  38  61  12
97  63  61   4  35
98  51  54  92  39
99  85  87  15  88

[100 rows x 4 columns]



In [15]:
# Bin column A based on fixed values and assign labels

df_num['Cut'] = pd.cut(df_num['A'], bins=[0, 25, 50, 75, 100], labels=["0-25", "25-50", "50-75", "75-100"], duplicates='drop')
print(df_num)

     A   B   C   D     Cut
0   23  29  37  92    0-25
1   74  31  23  71   50-75
2   19  26   0  71    0-25
3   58  67  37  77   50-75
4   33  21  61  33   25-50
..  ..  ..  ..  ..     ...
95  73  86  26  54   50-75
96  82  38  61  12  75-100
97  63  61   4  35   50-75
98  51  54  92  39   50-75
99  85  87  15  88  75-100

[100 rows x 5 columns]


In [20]:
# Bin column A based on the number of intervals and assign labels

df_num['BinsCut'] = pd.cut(df_num['A'], 6, labels=["B1", "B2", "B3", "B4", "B5","B6"])
print(df_num)

     A   B   C   D     Cut Qcut BinsCut
0   23  29  37  92    0-25   P2      B2
1   74  31  23  71   50-75   P3      B5
2   19  26   0  71    0-25   P1      B2
3   58  67  37  77   50-75   P3      B4
4   33  21  61  33   25-50   P2      B2
..  ..  ..  ..  ..     ...  ...     ...
95  73  86  26  54   50-75   P3      B5
96  82  38  61  12  75-100   P4      B5
97  63  61   4  35   50-75   P3      B4
98  51  54  92  39   50-75   P3      B4
99  85  87  15  88  75-100   P4      B6

[100 rows x 7 columns]


In [21]:
# Bin column A based on percentiles and assign labels
## Alternatively, pass the q parameter: 10 for deciles (10th percentiles), 4 for quartiles (4th percentiles).

df_num['Qcut'] = pd.qcut(df_num['A'], [0, 0.25, 0.5, 0.75, 1], labels=["P1", "P2", "P3", "P4"])
df_num['Qtilecut'] = pd.qcut(df_num['A'], 4, labels=["Q1", "Q2", "Q3", "Q4"])
print(df_num)


     A   B   C   D     Cut Qcut BinsCut Qtilecut
0   23  29  37  92    0-25   P2      B2       Q2
1   74  31  23  71   50-75   P3      B5       Q3
2   19  26   0  71    0-25   P1      B2       Q1
3   58  67  37  77   50-75   P3      B4       Q3
4   33  21  61  33   25-50   P2      B2       Q2
..  ..  ..  ..  ..     ...  ...     ...      ...
95  73  86  26  54   50-75   P3      B5       Q3
96  82  38  61  12  75-100   P4      B5       Q4
97  63  61   4  35   50-75   P3      B4       Q3
98  51  54  92  39   50-75   P3      B4       Q3
99  85  87  15  88  75-100   P4      B6       Q4

[100 rows x 8 columns]


### Sorting: sort_index, sort_values

Use the sort_index and sort_values methods. The former sorts by index, while the latter sorts by values (you need to specify the column).


In [11]:
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [25, 30, 35, 40, 45],
    'City': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'],
    'Salary': [60000, 70000, 80000, 90000, 100000]
}

df = pd.DataFrame(data, index=['A', 'B', 'C', 'D', 'E'])

print(df)

print(df)
print()
print(df.sort_index())
print()
print(df.sort_values(by="Salary", ascending=False))

      Name  Age         City  Salary
A    Alice   25     New York   60000
B      Bob   30  Los Angeles   70000
C  Charlie   35      Chicago   80000
D    David   40      Houston   90000
E      Eve   45      Phoenix  100000

      Name  Age         City  Salary
A    Alice   25     New York   60000
B      Bob   30  Los Angeles   70000
C  Charlie   35      Chicago   80000
D    David   40      Houston   90000
E      Eve   45      Phoenix  100000

      Name  Age         City  Salary
E      Eve   45      Phoenix  100000
D    David   40      Houston   90000
C  Charlie   35      Chicago   80000
B      Bob   30  Los Angeles   70000
A    Alice   25     New York   60000


### reshape

#### concat、merge

For DataFrames of different sizes, be sure to use the reset_index() method to reset the index.

1. concat: Concatenate along rows or columns.
2. merge: SQL-style JOIN.


In [32]:
# Simple concatenation using concat
# Has an axis parameter
# Pass a list as input

df = pd.DataFrame(np.random.randn(10, 4))
dflits = [df[:3].reset_index(), df[3:7].reset_index(), df[7:].reset_index()]
pprint(dflits)
print()
print(pd.concat(dflits, axis=1))

[   index         0         1         2         3
0      0  1.084744  0.004355  1.094328  0.371122
1      1 -1.171711 -0.047299 -0.380557 -1.729807
2      2 -0.606242 -0.057129  0.600145 -0.557517,
    index         0         1         2         3
0      3 -0.230784  0.307246  0.196767 -0.366682
1      4 -0.048933 -1.360850 -0.798853  0.278918
2      5 -0.943513 -0.790354 -0.534089  0.129146
3      6  1.146584 -1.523596 -2.218124  1.549359,
    index         0         1         2         3
0      7  0.929279 -0.157374 -1.558631 -0.203821
1      8 -0.673927  0.143456 -0.445237 -0.664387
2      9 -1.462854 -0.629377  0.504479  0.142350]

   index         0         1         2         3  index         0         1  \
0    0.0  1.084744  0.004355  1.094328  0.371122      3 -0.230784  0.307246   
1    1.0 -1.171711 -0.047299 -0.380557 -1.729807      4 -0.048933 -1.360850   
2    2.0 -0.606242 -0.057129  0.600145 -0.557517      5 -0.943513 -0.790354   
3    NaN       NaN       NaN       NaN  

In [8]:
# merge
## Equivalent to SQL JOIN

left = pd.DataFrame({"key": ["foo", "foo"], "lval": [1, 2]})
print(left)

right = pd.DataFrame({"key": ["foo", "foo"], "rval": [4, 5]})
print(right)

pd.merge(left, right, on="key")

   key  lval
0  foo     1
1  foo     2
   key  rval
0  foo     4
1  foo     5


,key,lval,rval
0,foo,1,4
1,foo,1,5
2,foo,2,4
3,foo,2,5


#### stack、pivot
Reshape consists of two parts:
1. stack: Converts column indices into row indices, can be reverted using unstack().
2. pivot_table(): Pivot table.


In [14]:
# example 
df = pd.DataFrame(
{
"A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
"B": ["one", "one", "two", "three", "two", "two", "one", "three"],
"C": np.random.randn(8),
"D": np.random.randn(8), }
)
print(df)

     A      B         C         D
0  foo    one  0.101200  0.248596
1  bar    one -1.207116 -1.424039
2  foo    two -0.493197  0.697662
3  bar  three -0.451021  2.915910
4  foo    two -0.521250  0.059482
5  bar    two -2.039441 -1.851153
6  foo    one  0.385903  0.370451
7  foo  three -1.043653 -0.214073


In [33]:
# reshape
## stack
arrays = [
   ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
   ["one", "two", "one", "two", "one", "two", "one", "two"],
]

print(df)
print()
index = pd.MultiIndex.from_arrays(arrays, names=["first", "second"])
df = pd.DataFrame(np.random.randn(8, 2), index=index, columns=["A", "B"])
df2 = df[:4]
print(df2)
print()

stacked = df2.stack(future_stack=True)
print(stacked)

                     A         B
first second                    
bar   one     0.529099  0.337105
      two    -1.126493 -1.369664
baz   one    -0.749937  1.312761
      two    -0.720214 -0.883736
foo   one    -0.929833  0.190915
      two    -0.595346 -1.426784
qux   one     1.410536 -0.760368
      two     0.076673  1.608456

                     A         B
first second                    
bar   one    -0.762222 -0.630633
      two    -1.306164 -0.190573
baz   one    -0.175900  0.217843
      two     1.256792 -0.986489

first  second   
bar    one     A   -0.762222
               B   -0.630633
       two     A   -1.306164
               B   -0.190573
baz    one     A   -0.175900
               B    0.217843
       two     A    1.256792
               B   -0.986489
dtype: float64


In [23]:
## pivot_table

df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 3,
        "B": ["A", "B", "C"] * 4,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 2,
        "D": np.random.randn(12),
        "E": np.random.randn(12),
    }
)
print(df)
pd.pivot_table(df, values="D", index=["A", "B"], columns=["C"])


        A  B    C         D         E
0     one  A  foo  0.123088 -0.164807
1     one  B  foo -0.926776 -0.542652
2     two  C  foo -0.979414 -0.758513
3   three  A  bar -0.042990  0.900497
4     one  B  bar -1.604740  1.805607
5     one  C  bar -0.316301 -0.684537
6     two  A  foo  0.418688  0.034220
7   three  B  foo  0.393327  0.925181
8     one  C  foo -0.275877  1.232546
9     one  A  bar  0.648915  0.238873
10    two  B  bar  2.929454  0.841834
11  three  C  bar -0.785253  1.370047


C             bar       foo
A     B                    
one   A  0.648915  0.123088
      B -1.604740 -0.926776
      C -0.316301 -0.275877
three A -0.042990       NaN
      B       NaN  0.393327
      C -0.785253       NaN
two   A       NaN  0.418688
      B  2.929454       NaN
      C       NaN -0.979414

#### groupby

In [40]:
# groupby
df = pd.DataFrame(
    {
        "A": ["foo", "bar", "foo", "bar", "foo", "bar", "foo", "foo"],
        "B": ["one", "one", "two", "three", "two", "two", "one", "three"],
        "C": np.random.randn(8),
        "D": np.random.randn(8),
    }
)

print(df)
print()
df_grouped = df.copy().groupby("A")[["D","C"]].sum()
print(df_grouped)

     A      B         C         D
0  foo    one  1.355561 -0.144665
1  bar    one -0.785882  0.505851
2  foo    two  0.759317 -0.839268
3  bar  three -0.353689 -1.813407
4  foo    two -1.447453 -0.439173
5  bar    two -0.816562  0.801510
6  foo    one -0.838415 -0.255706
7  foo  three -1.442524 -0.518148

            D         C
A                      
bar -0.506046 -1.956134
foo -2.196961 -1.613513


In [42]:
# After grouping, use transform to assign the data back to the corresponding records in the original DataFrame.

df[["new D","new C"]] = df.groupby("A")[["D","C"]].transform("sum")
df

,A,B,C,D,new D,new C
0,foo,one,1.355561,-0.144665,-2.196961,-1.613513
1,bar,one,-0.785882,0.505851,-0.506046,-1.956134
2,foo,two,0.759317,-0.839268,-2.196961,-1.613513
3,bar,three,-0.353689,-1.813407,-0.506046,-1.956134
4,foo,two,-1.447453,-0.439173,-2.196961,-1.613513
5,bar,two,-0.816562,0.801510,-0.506046,-1.956134
6,foo,one,-0.838415,-0.255706,-2.196961,-1.613513
7,foo,three,-1.442524,-0.518148,-2.196961,-1.613513


## Index and Column Names
### Multi-Level Index (MultiIndex)

Use `pd.MultiIndex.from_tuples()` or `pd.MultiIndex.from_product()` to create multi-level indexes.  
You can also use `pd.MultiIndex.from_array()`, which is a more intuitive approach.

1. The same principle applies: single brackets `[]` represent a single object, while double brackets `[[]]` represent multiple objects.
2. It is generally recommended to use `pd.MultiIndex.from_product()` as this generates multi-level indexes based on the **Cartesian product**.
3. If you need to customize the assignment of each sub-level label, you can use `pd.MultiIndex.from_tuples()`, passing a list of tuples representing the relationships.
4. The method for constructing the left-side index is similar: pass column names to `columns` and row names to `index`.
5. Remember to name the levels—this is a good habit.
6. `pd.MultiIndex.from_array()` takes an array for each level, for example: `pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b', 'b'], [1, 2, 1, 2, 1]])`. The tuple relationships are `('a', 1), ('a', 2), ('b', 1), ('b', 2), ('b', 1)`.


In [9]:
# product method

print(df)

myIndex = pd.MultiIndex.from_product([["Above A", "Above B"],["A", "B"]], names=["First", "Second"])

df.set_axis(myIndex, axis=1)

     A      B         C         D
0  foo    one  0.539148 -2.407817
1  bar    one -1.226543  0.194627
2  foo    two -0.197548 -0.659420
3  bar  three  1.314352 -0.128212
4  foo    two  0.272486 -1.765139
5  bar    two  0.732187 -2.352811
6  foo    one -1.038726 -0.506145
7  foo  three  1.625878 -0.446183


First  Above A          Above B          
Second       A      B         A         B
0          foo    one  0.539148 -2.407817
1          bar    one -1.226543  0.194627
2          foo    two -0.197548 -0.659420
3          bar  three  1.314352 -0.128212
4          foo    two  0.272486 -1.765139
5          bar    two  0.732187 -2.352811
6          foo    one -1.038726 -0.506145
7          foo  three  1.625878 -0.446183

In [14]:
# tuple method

df.columns = pd.MultiIndex.from_tuples([("Above A", "A"), ("Above A", "B"), ("Above B", "C"), ("Above B", "D")], names=["First", "Second"])
df

First  Above A          Above B          
Second       A      B         C         D
0          foo    one  0.539148 -2.407817
1          bar    one -1.226543  0.194627
2          foo    two -0.197548 -0.659420
3          bar  three  1.314352 -0.128212
4          foo    two  0.272486 -1.765139
5          bar    two  0.732187 -2.352811
6          foo    one -1.038726 -0.506145
7          foo  three  1.625878 -0.446183

### Interval Index (IntervalIndex)

Similar to MultiIndex, you can construct it using the corresponding methods.  
Apply it to the `index` parameter.

1. `pd.IntervalIndex.from_arrays`: Provide the arrays for the lower and upper bounds.
2. `pd.IntervalIndex.from_tuples`: Provide a list of tuples.
3. `pd.interval_range`: Provides the interval range.
4. `pd.IntervalIndex.from_breaks`: Provide the breakpoints.
5. Use `closed=` to set the closure type, the default is `'right'` (left-open, right-closed).
   1. `"right"`: Right-closed.
   2. `"left"`: Left-closed.
   3. `"both"`: Closed on both sides.
   4. `"neither"`: Open on both sides.


In [ ]:
# Arrays

# Start and end arrays
start = [1, 2, 3, 4]
end = [2, 3, 4, 5]

# Create IntervalIndex
interval_index = pd.IntervalIndex.from_arrays(left=start, right=end, closed='right')

print(interval_index)


In [ ]:
# tuples
## More Intuitive

intervals = [(1, 2), (2, 3), (3, 4), (4, 5)]

interval_index = pd.IntervalIndex.from_tuples(intervals, closed='right')

print(interval_index)


In [ ]:
# Equal Intervals

interval_index = pd.interval_range(start=0, end=5, periods=5, closed='right')

print(interval_index)


In [ ]:
# Breakpoints
# Can be used to create interval indices on existing data

# Create a simple DataFrame
df = pd.DataFrame({'Data': range(10)})

# First, create a stepped range
breaks = range(0, len(df) + 2, 2)

# Use these breakpoints (range) to create an IntervalIndex
interval_index = pd.IntervalIndex.from_breaks(breaks, closed='left')

# Set the new IntervalIndex as the index for the DataFrame
df.index = interval_index

print(df)


### Resetting Attributes: rename, set_index, reset_index

1. If you have a DataFrame, these are the methods to modify it directly.
2. Column names:
   1. `df.columns = ['a', 'b', 'c']`
   2. `df.rename(columns={'A': 'a', 'B': 'b'}, inplace=True)` requires passing a dictionary, and the length doesn't need to match.
   3. `df.set_axis(['a', 'b', 'c'], axis=1, inplace=True)` requires the length to match.
3. Index:
   1. `df.index = ['a', 'b', 'c']`
   2. `df.set_index('A', inplace=True)` sets an existing column as the index.
   3. `df.reset_index(inplace=True)` resets the index.
   4. `df.reindex(['a', 'b', 'c'])` reindexes the DataFrame, and missing values will be accepted.


## String Operations

Use by appending `str`, e.g., `df['A'].str.replace('a', 'b')`.


## Statistics and Calculations

#### EDA info()、describe()

In [6]:
print(df.dtypes)
print()
print(df.info())
print()
df.describe()

Name      object
Age        int64
City      object
Salary     int64
dtype: object

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, A to E
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    5 non-null      object
 1   Age     5 non-null      int64 
 2   City    5 non-null      object
 3   Salary  5 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 200.0+ bytes
None



,Age,Salary
count,5.000000,5.000000
mean,35.000000,80000.000000
std,7.905694,15811.388301
min,25.000000,60000.000000
25%,30.000000,70000.000000
50%,35.000000,80000.000000
75%,40.000000,90000.000000
max,45.000000,100000.000000


## Time Series


### Date: date_range

Accepts datetime objects.


In [46]:
# First, generate a pandas time index. For usage of the freq parameter, refer to the official documentation.

rng = pd.date_range("1/1/2024", periods=100, freq="D")

print(rng[:3])
print()

## Generate a time series based on business days, commonly used
### You can use datetime to specify start and end
### The freq parameter can be an instance of a DateOffset object. Refer to the official documentation for customization.

#### Common example: three_second_offset = pd.DateOffset(seconds=3)

start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2024, 1, 10)

rngb = pd.date_range(start=start, end=end, freq="B")
print(rngb[:6])


DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03'], dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04',
               '2024-01-05', '2024-01-08'],
              dtype='datetime64[ns]', freq='B')


In [ ]:
## Use this time series as the index to generate a series

ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)
print(ts.head(3))
print()

## Add a timezone to the series
ts = ts.tz_localize("UTC")
print(ts.head(3))
print()

## Change the timezone
ts = ts.tz_convert("US/Eastern")
print(ts.head(3))


In [11]:
## Apply business day offset to rng

rng = rng + pd.offsets.BusinessDay(5)
rng[:3]

DatetimeIndex(['2024-01-08', '2024-01-09', '2024-01-10'], dtype='datetime64[ns]', freq=None)

### Convert the time column to an index using to_datetime


In [19]:
data = {
    'Date': [20231011, 20231012, 20231013, 20231014, 20231015, 20231016],
    'Value': range(6)
}
df_int_date = pd.DataFrame(data)
print(df_int_date)

       Date  Value
0  20231011      0
1  20231012      1
2  20231013      2
3  20231014      3
4  20231015      4
5  20231016      5


In [21]:
# Convert the date column to a datetime index
## Use the to_datetime method and specify the format parameter.

df_int_date['Date'] = pd.to_datetime(df_int_date['Date'], format='%Y%m%d')
df_int_date.set_index(df_int_date['Date'], inplace=True)
df_int_date.drop(columns='Date', inplace=True)
df_int_date

,Value
Date,
2023-10-11,0
2023-10-12,1
2023-10-13,2
2023-10-14,3
2023-10-15,4
2023-10-16,5


### Basic Time Series Operations: resample


In [48]:
# resample
## Grouping operation for time series
## Pay attention to the opening and closing of time intervals; refer to the official documentation
## Passing parameters intuitively is generally fine, you can pass a dateoffset object. Again, if unsure, refer to the official documentation.

start = datetime.datetime(2024, 1, 1)
end = datetime.datetime(2024, 3, 10)

rngb = pd.date_range(start = start, end = end, freq="B")

ts = pd.Series(np.random.randint(0, 500, len(rngb)), index=rngb)
print(ts.head(3))
print()

print(ts.resample("1d").sum())
print()
print(ts.resample("5d").ohlc())

2024-01-01    419
2024-01-02    271
2024-01-03    151
Freq: B, dtype: int32

2024-01-01    419
2024-01-02    271
2024-01-03    151
2024-01-04    443
2024-01-05    251
             ... 
2024-03-04    109
2024-03-05    387
2024-03-06    350
2024-03-07    431
2024-03-08    257
Freq: D, Length: 68, dtype: int32

            open  high  low  close
2024-01-01   419   443  151    251
2024-01-06   331   331  207    207
2024-01-11   149   445  149    407
2024-01-16   377   472   73     73
2024-01-21   454   454   13    172
2024-01-26    92   180   92    180
2024-01-31    84   394   84    112
2024-02-05   491   499   20    107
2024-02-10   480   480  451    465
2024-02-15   483   483   75     75
2024-02-20   223   321  173    277
2024-02-25   480   490   19    184
2024-03-01   309   387  109    387
2024-03-06   350   431  257    257


In [51]:
# Using resample to cause upsampling
## Splitting the time series into smaller intervals, effectively increasing the frequency of the time series, resulting in NaN values in between
print(ts.resample("12H").asfreq())

## Use ffill to fill missing values
### You can set the limit parameter to restrict the number of fills
### You can set the limit_area parameter to restrict the fill range
print()
print(ts.resample("12H").asfreq().ffill())


2024-01-01 00:00:00    419.0
2024-01-01 12:00:00      NaN
2024-01-02 00:00:00    271.0
2024-01-02 12:00:00      NaN
2024-01-03 00:00:00    151.0
                       ...  
2024-03-06 00:00:00    350.0
2024-03-06 12:00:00      NaN
2024-03-07 00:00:00    431.0
2024-03-07 12:00:00      NaN
2024-03-08 00:00:00    257.0
Freq: 12H, Length: 135, dtype: float64

使用ffill填充
2024-01-01 00:00:00    419.0
2024-01-01 12:00:00    419.0
2024-01-02 00:00:00    271.0
2024-01-02 12:00:00    271.0
2024-01-03 00:00:00    151.0
                       ...  
2024-03-06 00:00:00    350.0
2024-03-06 12:00:00    350.0
2024-03-07 00:00:00    431.0
2024-03-07 12:00:00    431.0
2024-03-08 00:00:00    257.0
Freq: 12H, Length: 135, dtype: float64


### Holiday Handling: CustomBusinessDay

Refer to [Holiday Calendar](https://pandas.pydata.org/docs/user_guide/timeseries.html#holidays-holiday-calendars) for custom handling of holidays.


In [53]:
# Simple Customization: CustomBusinessDay
## Based on U.S. federal holidays
## For Egypt, since the workdays are Sunday to Thursday, customize the weekmask parameter
## Then pass in the holidays parameter

myb = CustomBusinessDay(weekmask='Sun Mon Tue Wed Thu', holidays=['2018-04-15'])
pd.date_range(start='4/1/2018', end='4/30/2018', freq=myb)

In [52]:
# Custom Calendar, example provided here only

from pandas.tseries.holiday import (
    Holiday,
    USMemorialDay,
    AbstractHolidayCalendar,
    nearest_workday,
)
from pandas.tseries.offsets import CustomBusinessDay, Day


class ExampleCalendar(AbstractHolidayCalendar):
    rules = [
        USMemorialDay,

        ## Custom holiday rules
        ### The observance or offset parameter is used to adjust holidays to the nearest working day
        ### If holidays fall on Saturday or Sunday, they will be adjusted to a weekday to avoid losing the holiday.

        Holiday("My Birthday", month=7, day=4, observance=nearest_workday),
        
        Holiday(
            "Columbus Day",
            month=10,
            day=1,
            offset=pd.DateOffset(weekday=Day(2)),
        ),
    ]

# Instantiate a custom business day and use it

my_custom_bday = CustomBusinessDay(calendar=ExampleCalendar())
pd.date_range(start="1/1/2024", end="1/10/2024", freq=my_custom_bday)

## Rolling Window

The rolling() method can be used not only for time series but for any sequence.  
Refer to (https://pandas.pydata.org/docs/user_guide/window.html)

1. The `win_type` parameter can specify the type of weighted window, such as 'triang', and it accepts scipy window types, like `gaussian`.
   - After passing `win_type`, methods like `mean()` will behave differently, with the default being an equal-weighted window.
   - If the mathematical method itself requires more parameters, pandas’ built-in methods like `mean` may raise errors.
   - For example, when using `win_type='gaussian'`, you need to apply `.apply(lambda x: np.mean(x), std=2)`.
2. Applying rolling to time series allows you to use methods like corr() on the series.


In [54]:
# simple rolling
s = pd.Series(range(5))
print(s)

print(s.rolling(window=3).sum())

0    0
1    1
2    2
3    3
4    4
dtype: int64
0    NaN
1    NaN
2    3.0
3    6.0
4    9.0
dtype: float64


In [59]:
# Iteration with rolling

for i in s.rolling(window=3):
    print(i)


2020-01-01    0
Freq: D, dtype: int64
2020-01-01    0
2020-01-02    1
Freq: D, dtype: int64
2020-01-01    0
2020-01-02    1
2020-01-03    2
Freq: D, dtype: int64
2020-01-02    1
2020-01-03    2
2020-01-04    3
Freq: D, dtype: int64
2020-01-03    2
2020-01-04    3
2020-01-05    4
Freq: D, dtype: int64


In [60]:
# Specifying a central window

for i in s.rolling(window=3, center=True):
    print(i)

2020-01-01    0
2020-01-02    1
Freq: D, dtype: int64
2020-01-01    0
2020-01-02    1
2020-01-03    2
Freq: D, dtype: int64
2020-01-02    1
2020-01-03    2
2020-01-04    3
Freq: D, dtype: int64
2020-01-03    2
2020-01-04    3
2020-01-05    4
Freq: D, dtype: int64
2020-01-04    3
2020-01-05    4
Freq: D, dtype: int64


In [56]:
# Windows can specify dateoffset

s = pd.Series(range(5), index=pd.date_range('2020-01-01', periods=5, freq='1D'))

s.rolling(window='2D').sum()

2020-01-01    0.0
2020-01-02    1.0
2020-01-03    3.0
2020-01-04    5.0
2020-01-05    7.0
Freq: D, dtype: float64

In [57]:
# By specifying method='table' and using the apply method, complex rolling operations can be achieved
## When method is 'table', the engine must be set to numba
## Below is an example
### Extremely slow performance

def weighted_mean(x):
    arr = np.ones((1, x.shape[1]))
    arr[:, :2] = (x[:, :2] * x[:, 2]).sum(axis=0) / x[:, 2].sum()
    return arr

df = pd.DataFrame([[1, 2, 0.6], [2, 3, 0.4], [3, 4, 0.2], [4, 5, 0.7]])

df.rolling(2, method="table", min_periods=0).apply(weighted_mean, raw=True, engine="numba")

,0,1,2
0,1.000000,2.000000,1.0
1,1.800000,2.000000,1.0
2,3.333333,2.333333,1.0
3,1.555556,7.000000,1.0


## Use Case
### Create a difference sequence


In [62]:
# Create example Series
s = pd.Series([1, 2, 4, 7, 11])

# Calculate first-order difference
diff_series = s.diff(periods=1)

print(diff_series)


0    NaN
1    1.0
2    2.0
3    3.0
4    4.0
dtype: float64


### Filling Values, Including Filling with 0

1. Using custom rules can prevent strings' NaN values from being replaced, commonly used.
2. However, fillna() offers more filling methods. You can specify the `method=` parameter, such as `ffill` (forward fill) or `bfill` (backward fill), for forward or backward filling.


In [10]:
# Create an example DataFrame containing NaN values

df = pd.DataFrame({
    'numeric_column': [1, np.nan, 3],
    'string_column': ['apple', np.nan, 'banana']
})
print(df)

   numeric_column string_column
0             1.0         apple
1             NaN           NaN
2             3.0        banana


In [11]:
# Define a custom function to decide the fill value based on the column's data type
def fillna_custom(x):
    if x.dtype == float or x.dtype == int:
        return x.fillna(0)
    else:
        return x

# Apply the custom function using the apply method
df_customfill = df.apply(fillna_custom)

print(df_customfill)


   numeric_column string_column
0             1.0         apple
1             0.0           NaN
2             3.0        banana


In [12]:
# Or simply use fillna()
df = pd.DataFrame({
    'numeric_column': [1, np.nan, 3],
    'string_column': ['apple', np.nan, 'banana']
})
print(df)

df.fillna(0, inplace=True)
print(df)


   numeric_column string_column
0             1.0         apple
1             NaN           NaN
2             3.0        banana
   numeric_column string_column
0             1.0         apple
1             0.0             0
2             3.0        banana


### Interpolation

Use the `interpolate()` method.

1. 'linear': Linear interpolation.
2. 'time': Time-based interpolation (particularly useful for time indices).
3. 'index', 'values': Linear interpolation using the index.
4. 'pad', 'ffill': Forward fill.
5. 'bfill', 'backfill': Backward fill.
6. 'nearest': Nearest neighbor interpolation.
7. 'polynomial': Polynomial interpolation, requires specifying the order (degree of the polynomial).
8. 'spline': Spline interpolation, requires specifying the order (degree of the spline).
9. Or specify scipy's interpolation methods, such as 'quadratic', 'cubic', etc.


In [3]:

# Example DataFrame
df = pd.DataFrame({
    'ID': range(1, 6),
    'Temperature': [22.5, 21.0, np.nan, 23.5, 24.0],
    'Humidity': [60, np.nan, 65, 70, np.nan],
    'Location': ['Room1', 'Room2', 'Room1', 'Room2', np.nan]
})

print(df)


   ID  Temperature  Humidity Location
0   1         22.5      60.0    Room1
1   2         21.0       NaN    Room2
2   3          NaN      65.0    Room1
3   4         23.5      70.0    Room2
4   5         24.0       NaN      NaN


In [6]:
# The limit parameter restricts the number of consecutive interpolations.

df["Temperature"].interpolate(method="linear", inplace=True,limit=1)
print(df)

   ID  Temperature  Humidity Location
0   1        22.50      60.0    Room1
1   2        21.00       NaN    Room2
2   3        22.25      65.0    Room1
3   4        23.50      70.0    Room2
4   5        24.00       NaN      NaN


### Remove Duplicate Rows and Columns


In [14]:
import pandas as pd

# Example DataFrame
df = pd.DataFrame({
    'A': [1, 1, 2, 2, 3],
    'B': [1, 1, 2, 2, 3],
    'C': ['x', 'x', 'y', 'z', 'z']
})
print(df)


   A  B  C
0  1  1  x
1  1  1  x
2  2  2  y
3  2  2  z
4  3  3  z


In [10]:
# Remove duplicate rows
# Use keep='first' to remove the first occurrence of duplicate rows, and use keep='last' to remove the last occurrence.
# Use keep=False to remove all duplicate rows.

df = df.drop_duplicates(keep='first')

print(df)

   A  B  C
0  1  1  x
2  2  2  y
3  2  3  z
4  3  3  z


In [15]:
# Remove duplicate columns
# Transpose the DataFrame, remove duplicate rows (which now correspond to the original columns), and then transpose it back.

df = df.T.drop_duplicates().T

print(df)


   A  C
0  1  x
1  1  x
2  2  y
3  2  z
4  3  z


### Remove Invalid Values

Use the dropna() method.

1. Common parameters include axis, how, thresh, subset, and inplace.
2. axis: 0 indicates row deletion, 1 indicates column deletion. Default is 0.
3. how: 'any' means delete if any NaN exists, 'all' means delete only if all values are NaN. Default is 'any'.
4. thresh: Integer, retains rows with at least thresh non-NaN values.
5. subset: List, specifies columns in which to check for NaN.
6. inplace: Boolean, determines whether to modify the original data in place.


In [ ]:
import pandas as pd
import numpy as np

# Create a DataFrame containing NaN values
data = {'Name': ['Alex', 'Bob', 'Clarke', 'Dave'],
        'Age': [25, np.nan, 22, 30],
        'Gender': ['Male', 'Male', np.nan, 'Male']}
df = pd.DataFrame(data)

print("Original DataFrame:")
print(df)

# Use dropna() to delete rows that contain any NaN values
df_cleaned = df.dropna()

print("\nDataFrame after dropping rows with NaN values:")
print(df_cleaned)


## Key Points 

1. General rule: **Single value** uses single brackets, **multiple values** use double brackets.
2. Wherever an index can be passed, slicing is generally allowed.
3. Both names and numbers can be used for slicing.
4. Behavior of df[]:
   - To select a column, use the column name, e.g., `df['A']` will select column A and return a series; `df[['A', 'B']]` will select columns A and B and return a DataFrame.
   - To select rows, you can either use slicing or pass a boolean series; e.g., `df[df['A'] > 0]` will return all rows where column A is greater than 0.
   - If you pass a boolean DataFrame, like `df[df > 0]`, it will return a DataFrame where all places that don’t meet the condition are filled with NaN.
5. apply() is the most commonly used method for custom operations. For DataFrames, `x` defaults to operating on columns; for series, `x` defaults to operating on elements.
   - applymap() applies `x` to each element of the DataFrame by default.
